In [1]:
import tensorflow as tf
import torch
from transformers import GPT2Model, GPT2Config
import ipywidgets
from IPython import display

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)


Running on device:  cuda


In [4]:
# Initializing a GPT2 configuration
configuration = GPT2Config()
# Initializing a model from the configuration
model = GPT2Model(configuration)
# Accessing the model configuration
configuration = model.config
print(configuration)

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "vocab_size": 50257
}



In [5]:
from transformers import AutoTokenizer, AutoModelWithLMHead

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2-medium')
# Download model and configuration from S3 and cache.
model = AutoModelWithLMHead.from_pretrained('gpt2-medium', pad_token_id=tokenizer.eos_token_id)


input_context = 'Eating disorders are'
bad_words_ids = [tokenizer.encode(bad_word, add_prefix_space=True) for bad_word in ['idiot', 'stupid', 'shut up']]
input_ids = tokenizer.encode(input_context, return_tensors='pt')  # encode input context

c:\users\carson\onedrive\desktop\programming\projects\epic\transformers\src\transformers\modeling_auto.py:810: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [6]:
# Normal Output
outputs = model.generate(input_ids=input_ids, max_length=100, do_sample=True, bad_words_ids=bad_words_ids)  # generate sequences without allowing bad_words to be generated
print("Normal Output:\n" + 100 * '-')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Normal Output:
----------------------------------------------------------------------------------------------------
Eating disorders are not typically listed as a treatable condition in the Diagnostic and Statistical Manual of Mental Disorders (DSM-5).

How is eating disorder care coordinated?

Eating disorder therapy is coordinated by a professional trained in the management of eating disorders. The type of eating disorder is determined by the individual and his or her presentation of DSM-IV eating disorders. Eating disorders are classified into three categories:

Overweight and underweight – this group may be classified either


In [7]:
# activate beam search and early_stopping
beam_output = model.generate(
    input_ids, 
    max_length=100, 
    num_beams=5, 
    early_stopping=True
)

print("Beam Search Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))


Beam Search Output:
----------------------------------------------------------------------------------------------------
Eating disorders are the most common form of eating disorders in the United States, accounting for more than one-third of all eating disorders reported to the National Institute of Mental Health (NIMH), according to the National Eating Disorders Association (NEDA).

The NEDA defines an eating disorder as:

"Anorexia nervosa, bulimia nervosa, binge eating disorder, and anorexia nervosa plus anorexia nervosa plus bulimia nerv


In [8]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    input_ids, 
    max_length=100, 
    num_beams=6, 
    no_repeat_ngram_size=6, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Eating disorders are the most common form of eating disorders in the United States, accounting for more than one-third of all eating disorders reported to the National Institute of Mental Health (NIMH), according to the National Eating Disorders Association (NEDA).

According to the NEDA, eating disorders are the most common type of eating disorder in the United States, with an estimated 1.3 million people in the U.S. suffering from an eating disorder in 2013.




In [9]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids, 
    max_length=100, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


Output:
----------------------------------------------------------------------------------------------------
0: Eating disorders are the most common form of eating disorders in the United States, accounting for more than one-third of all eating-disorder cases.1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
1: Eating disorders are the most common form of eating disorders in the United States, accounting for more than one-third of all eating-disorder-related hospitalizations.1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,
2: Eating disorders are the most common form of eating disorders in the United States, accounting for more than one-third of all eating-disorder cases.1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,32,33,34,35,36
3: Eating disorders are the most common form of eating disorders in the United States, accounting for more than one-third of all eating-

In [10]:
# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100, 
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Eating disorders are now a complex mixture of genes, eating attitudes, and situations. A binge-eating disorder (or BED) can develop due to extreme enough emotional stress or overeating that the body is unable to control itself or even learn to sustain stress. Since tonics in normal eating can have little or no acute effect, someone who doesn't properly adhere to diet alone will show signs of curdling when they restrict their diet to one meal a day until from time to time.



In [11]:
# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100, 
    top_k=0, 
    temperature=0.7
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Eating disorders are very common, and many people with them are at risk of developing other health problems, including obesity, type 2 diabetes, and heart disease.

What are the risks of food allergies?

If you have an allergy to one or more of the common foods listed above, you will likely experience some of the symptoms of food allergy, but most people will not experience any severe symptoms.

Table 1. Food allergy symptoms

a) Allergy symptoms that may be


In [12]:
# set top_k to 50
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100, 
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Eating disorders are becoming the norm, not the exception. Many individuals with eating disorders believe that eating disorders are something unique to their particular situation and situation is not the issue. However, in an effort to encourage awareness and access to education and treatment services related to eating disorders, it has become necessary to ask one person what is the most important thing in life.

This is a blog post for information on one or more particular mental illnesses and related disorders. However, it does not necessarily mean that


In [13]:
# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100, 
    top_p=0.92, 
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Eating disorders are common, but they are often determined, as in chronic fatigue syndrome, by factors outside the actions of your brain. The cause of two well-known conditions, irritable bowel syndrome and Crohn's disease, is not known; however, factors such as genetic makeup and diet are known to trigger some of these disorders.

One sure sign of irritable bowel syndrome, known as irritable bowel disease and ulcerative colitis, is constipation. It is caused by


In [14]:
# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=100, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: Eating disorders are also a common cause of anxiety and depression.

"While this study focuses on a population that is at higher risk, our findings suggest that a range of behaviors — from diet, exercise and environmental exposures — can be associated with anxiety and depression in eating disorders."

_____

Baeza R, Ojala A, Fava JH, Kao P, et al. Obesity and anxiety are comorbidities in eating disorders. J Psychosom
1: Eating disorders are caused by poor nutritional habits, which can include excessive sugar consumption and excessive exercise. One study found that obese people who consumed more calories than needed daily were 30 to 40 percent more likely to develop a metabolic syndrome.

Exercise training and eating disorders can occur because people simply don't know how to control their intake, said Dr. Robert Schwartz, the senior director of the division of metabolic and 

Worrying but funny output:


"My cute dog was so upset and he yelled at me! He came to try and find me!" she said.
Cecilia said she did not feel safe for about 2 hours until police came to her side. "I was screaming but I couldn't see because of the sun light," she said. "Then the police said there's nothing they could do. I kept saying my dog has to go but I don't believe that because I live right on top of them."


* min_length: can be used to force the model to not produce an EOS token (= not finish the sentence) before min_length is reached. This is used quite frequently in summarization, but can be useful in general if the user wants to have longer outputs.

* repetition_penalty: can be used to penalize words that were already generated or belong to the context. It was first introduced by Kesker et al. (2019) and is also used in the training objective in Welleck et al. (2019). It can be quite effective at preventing repetitions, but seems to be very sensitive to different models and use cases, e.g. see this discussion on Github.

* attention_mask: can be used to mask padded tokens

* pad_token_id, bos_token_id, eos_token_id: If the model does not have those tokens by default, the user can manually choose other token ids to represent them.

